In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Set up the notebook environment
!pip install catboost xgboost lightgbm scikit-learn
import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy.io
from scipy import stats
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.signal import butter, filtfilt, welch, decimate
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 3.8 MB/s eta 0:00:00


In [ ]:
data_train = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/Ted Yang/Copy of raw_training_data.mat')
data_leader = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/Ted Yang/Copy of leaderboard_data.mat')

In [ ]:
data_new_1 = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/additional data/BCICIV_4_mat/sub1_comp.mat')
data_new_2 = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/additional data/BCICIV_4_mat/sub2_comp.mat')
data_new_3 = scipy.io.loadmat('/content/drive/Shareddrives/BE 5210 Shared Drive/additional data/BCICIV_4_mat/sub3_comp.mat')

* Data Structure
* 6min, 40s, 400,000 samples, 1000Hz

In [ ]:
data_leader = data_leader['leaderboard_ecog']

In [ ]:
data_glove = data_train['train_dg']
data_ecog = data_train['train_ecog']
train_dg = []
test_dg = []
train_ecog = []
test_ecog = []

# Define the split ratio (e.g., 70% for training and 30% for testing)
split_ratio = 0.7

# Function to split each subject's data
def split_data(data, split_ratio):
    num_samples = len(data[0])
    split_point = int(num_samples * split_ratio)
    train = data[0][:split_point]
    test = data[0][split_point:]
    return train, test

# Loop over each subject's data for glove and ECoG data
for glove_data, ecog_data in zip(data_glove, data_ecog):
    train_dg_split, test_dg_split = split_data(glove_data, split_ratio)
    train_ecog_split, test_ecog_split = split_data(ecog_data, split_ratio)

    # Append the split data to their respective lists
    train_dg.append(train_dg_split)
    test_dg.append(test_dg_split)
    train_ecog.append(train_ecog_split)
    test_ecog.append(test_ecog_split)

# Now train_dg, test_dg, train_ecog, and test_ecog contain the split data for each subject

#training set
total_samples_train_ecog = len(train_ecog[0])

#testing set
total_samples_test_ecog = len(test_ecog[0])

#full dataset sample before the split
total_samples_full_ecog = total_samples_train_ecog + total_samples_test_ecog

print(f'The total number of samples is {total_samples_full_ecog}.')
print(f'The number of samples in the training set is {total_samples_train_ecog}.')
print(f'The number of samples in the testing set is {total_samples_test_ecog}.')

The total number of samples is 300000.
The number of samples in the training set is 210000.
The number of samples in the testing set is 90000.


In [ ]:
def filter_data(raw_eeg, fs=1000):
    """
    Input:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)

    Output:
      clean_data (samples x channels): the filtered signal
    """

    #lowcut and highcut for the bandpass filter
    lowcut = 0.15
    highcut = 200.0

    #Butterworth bandpass filter
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(4, [low, high], btype='band', analog=False)

    #apply filter to raw ECoG data
    clean_data = filtfilt(b, a, raw_eeg, axis=0)

    return clean_data


In [ ]:
#window and step sizes
window_length = 100
step_size = 50

#number of feature windows M for the full dataset, training, and testing sets
M_full = 1 + (total_samples_full_ecog - window_length) // step_size
M_train = 1 + (total_samples_train_ecog - window_length) // step_size
M_test = 1 + (total_samples_test_ecog - window_length) // step_size

print(f'There are {M_full} windows using all the data.')
print(f'There are {M_train} windows training data.')
print(f'There are {M_test} windows testing data.')

There are 5999 windows using all the data.
There are 4199 windows training data.
There are 1799 windows testing data.


# **Feature Extraction**

# This one has got 0.4426 for testing set

In [ ]:
"""#This one has got 0.4426 for testing set

def get_features(filtered_window, fs=1000):

    Inputs:
        filtered_window (window_samples x channels): the window of the filtered ECoG signal
        fs: sampling rate

    Output:
        features (channels x num_features): the features calculated on each channel for the window

    # Make sure nperseg does not exceed window size
    nperseg = min(filtered_window.shape[0], 256)

    # Basic statistical features
    mean = np.mean(filtered_window, axis=0)
    variance = np.var(filtered_window, axis=0)
    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential

    # Compute the Power Spectral Density (PSD) for each channel
    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)

    # Define frequency bands
    bands = {
        'alpha': (8, 12),
        #'beta': (13, 30),
        #'theta': (4, 7),  # Adding an extra band as an example
        #'low_gamma': (30, 50),  # Another example band
        'custom_band1': (18, 24),
        'custom_band2': (75, 115),
        'custom_band3': (125, 159),
        'custom_band4': (159, 175)
    }

    # Initialize a dictionary to hold the power for each band
    band_powers = {band: np.zeros(psd.shape[0]) for band in bands}

    # Calculate power for each band
    for band, (low, high) in bands.items():
        freq_mask = (freqs >= low) & (freqs <= high)
        band_powers[band] = np.sum(psd[:, freq_mask], axis=1)

    # Collect all features
    all_features = [mean, variance, lmp] + list(band_powers.values())
    features = np.vstack(all_features).T

    return features
"""

"#This one has got 0.4426 for testing set\n\ndef get_features(filtered_window, fs=1000):\n\n    Inputs:\n        filtered_window (window_samples x channels): the window of the filtered ECoG signal\n        fs: sampling rate\n\n    Output:\n        features (channels x num_features): the features calculated on each channel for the window\n\n    # Make sure nperseg does not exceed window size\n    nperseg = min(filtered_window.shape[0], 256)\n\n    # Basic statistical features\n    mean = np.mean(filtered_window, axis=0)\n    variance = np.var(filtered_window, axis=0)\n    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential\n\n    # Compute the Power Spectral Density (PSD) for each channel\n    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)\n\n    # Define frequency bands\n    bands = {\n        'alpha': (8, 12),\n        #'beta': (13, 30),\n        #'theta': (4, 7),  # Adding an extra band as an example\n        #'low_gamma': (30, 50),  # Another example 

In [ ]:
def get_features(filtered_window, fs=1000):
    """
    Inputs:
        filtered_window (window_samples x channels): the window of the filtered ECoG signal
        fs: sampling rate

    Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    # Make sure nperseg does not exceed window size
    nperseg = min(filtered_window.shape[0], 256)

    # Basic statistical features
    mean = np.mean(filtered_window, axis=0)
    variance = np.var(filtered_window, axis=0)
    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential

    # Compute the Power Spectral Density (PSD) for each channel
    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)

    # Define frequency bands
    bands = {
        'alpha': (8, 12),
        #'beta': (13, 30),
        #'theta': (4, 7),  # Adding an extra band as an example
        #'low_gamma': (30, 50),  # Another example band
        'custom_band1': (18, 24),
        'custom_band2': (75, 115),
        'custom_band3': (125, 159),
        'custom_band4': (159, 175)
    }

    # Initialize a dictionary to hold the power for each band
    band_powers = {band: np.zeros(psd.shape[0]) for band in bands}

    # Calculate power for each band
    for band, (low, high) in bands.items():
        freq_mask = (freqs >= low) & (freqs <= high)
        band_powers[band] = np.sum(psd[:, freq_mask], axis=1)

    # Collect all features
    all_features = [mean, variance, lmp] + list(band_powers.values())
    features = np.vstack(all_features).T

    return features



---




In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length_ms, window_overlap_ms):
    """
    Inputs:
        raw_ecog (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length_ms: the window's length in milliseconds
        window_overlap_ms: the window's overlap in milliseconds

    Output:
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
    """
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    #filter_data function placeholder
    filtered_ecog = raw_ecog

    total_samples = raw_ecog.shape[0]
    num_windows = 1 + (total_samples - window_length) // step_size

    feature_list = []

    for start_idx in tqdm(range(0, total_samples - window_length + 1, step_size)):
        end_idx = start_idx + window_length
        window = filtered_ecog[start_idx:end_idx, :]

        #features for this window
        window_features = get_features(window, fs)

        #append features to list
        feature_list.append(window_features.flatten())

    #stack to get 2D array (num_windows x features)
    all_feats = np.array(feature_list)

    return all_feats

In [ ]:
def create_R_matrix(features, N=3):
    """
    Input:
        features: The 2D array of features (num_windows x num_features_per_window).
        N: The number of previous windows to include for each prediction instance.

    Output:
        R: The response matrix.
    """
    #first N-1 rows to beginning of features matrix
    adjusted_features = np.vstack([features[:N-1], features])

    #number of total instances after adjustment
    M_prime = adjusted_features.shape[0] - (N - 1)

    #initialize response matrix R
    num_features = features.shape[1]
    R = np.zeros((M_prime, N * num_features + 1))  # +1 for the intercept term

    #fill in R
    for i in range(M_prime):
        #extract N consecutive windows of features
        consecutive_features = adjusted_features[i:i+N].flatten()
        #fill corresponding row in R, adding 1 as the last column
        R[i, 1:] = consecutive_features
        R[i, 0] = 1  # Intercept term

    return R

In [ ]:
train_ecog_1 = train_ecog[0]
train_ecog_2 = train_ecog[1]
train_ecog_3 = train_ecog[2]

train_glove_1 = train_dg[0]
train_glove_2 = train_dg[1]
train_glove_3 = train_dg[2]

In [ ]:
#feature extraction for each subject
feature_1 = get_windowed_feats(train_ecog_1, 1000, 100, 50)
feature_2 = get_windowed_feats(train_ecog_2, 1000, 100, 50)
feature_3 = get_windowed_feats(train_ecog_3, 1000, 100, 50)

num_target_windows = feature_1.shape[0]
num_target_windows

100%|██████████| 4199/4199 [00:08<00:00, 468.57it/s] 


4199

In [ ]:
#get R matrix
R_1 = create_R_matrix(feature_1, N=3)
R_2 = create_R_matrix(feature_2, N=3)
R_3 = create_R_matrix(feature_3, N=3)

In [ ]:
def get_windowed_target(glove_data, window_length_ms, window_overlap_ms, fs=1000):
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    num_windows = 1 + (glove_data.shape[0] - window_length) // step_size
    targets = []

    for start_idx in range(0, glove_data.shape[0] - window_length + 1, step_size):
        end_idx = start_idx + window_length
        window = glove_data[start_idx:end_idx, :]
        targets.append(np.mean(window, axis=0))  # Example: mean over the window

    return np.array(targets)

# Windowing target glove data
target_1 = get_windowed_target(train_glove_1, 100, 50)
target_2 = get_windowed_target(train_glove_2, 100, 50)
target_3 = get_windowed_target(train_glove_3, 100, 50)


In [ ]:
# Define parameter distributions for XGBoost
"""xgb_params = {
    'n_estimators': np.arange(100, 501, 50),
    'max_depth': np.arange(3, 10, 1),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'subsample': np.linspace(0.5, 1, 6),
    'colsample_bytree': np.linspace(0.5, 1, 6)
}"""

# Define parameter distributions for CatBoost
cat_params = {
    'iterations': np.arange(100, 501, 50),
    'depth': np.arange(3, 10, 1),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'l2_leaf_reg': np.linspace(1, 10, 10)
}

# Setup RandomizedSearchCV for XGBoost
"""xgb_random = RandomizedSearchCV(
    XGBRegressor(),
    param_distributions=xgb_params,
    n_iter=100,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)"""

# Setup RandomizedSearchCV for CatBoost
cat_random = RandomizedSearchCV(
    CatBoostRegressor(verbose=False),
    param_distributions=cat_params,
    n_iter=100,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the models (example for one dataset)
xgb_random.fit(R_1[:, 1:], target_1)
cat_random.fit(R_1[:, 1:], target_1)

# Print the best parameters and scores
print("Best parameters for XGBoost:", xgb_random.best_params_)
print("Best score for XGBoost:", xgb_random.best_score_)
print("Best parameters for CatBoost:", cat_random.best_params_)
print("Best score for CatBoost:", cat_random.best_score_)


In [ ]:
# XGBoost Model
xgb_model1 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model1.fit(R_1[:, 1:], target_1)

xgb_model2 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model2.fit(R_2[:, 1:], target_2)

xgb_model3 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model3.fit(R_3[:, 1:], target_3)

# CatBoost Model
cat_model1 = CatBoostRegressor(iterations=300, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model1.fit(R_1[:, 1:], target_1)

cat_model2 = CatBoostRegressor(iterations=300, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model2.fit(R_2[:, 1:], target_2)

cat_model3 = CatBoostRegressor(iterations=300, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model3.fit(R_3[:, 1:], target_3)

In [ ]:
# CatBoost Model
cat_model1 = CatBoostRegressor(iterations=500, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model1.fit(R_1[:, 1:], target_1)

cat_model2 = CatBoostRegressor(iterations=500, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model2.fit(R_2[:, 1:], target_2)

cat_model3 = CatBoostRegressor(iterations=500, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model3.fit(R_3[:, 1:], target_3)

----> Test

In [ ]:
"""#create linear f function
def linear_filter(R_matrix, train_data):
    num_samples_features = R_matrix.shape[0]
    num_samples_flexion = train_data.shape[0]

    #downsample train_data if it doesn't match the number of feature windows in R_matrix
    if num_samples_features != num_samples_flexion:
        downsample_factor = num_samples_flexion // num_samples_features

        #make nsure downsample_factor at least 1
        downsample_factor = max(downsample_factor, 1)
        downsize_data = decimate(train_data, downsample_factor, axis=0)

        #make sure downsize_data has same number of samples as R_matrix
        if downsize_data.shape[0] > num_samples_features:
            downsize_data = downsize_data[:num_samples_features]
    else:
        downsize_data = train_data

    #linear filter f using matrix multiplication
    #p1 = (R^T R)^(-1)
    p1 = np.linalg.inv(np.matmul(R_matrix.T, R_matrix))

    #p2 = R^T Y
    p2 = np.matmul(R_matrix.T, downsize_data)

    #f = p1 * p2
    lf = np.matmul(p1, p2)

    return downsize_data, lf"""

"#create linear f function\ndef linear_filter(R_matrix, train_data):\n    num_samples_features = R_matrix.shape[0]\n    num_samples_flexion = train_data.shape[0]\n\n    #downsample train_data if it doesn't match the number of feature windows in R_matrix\n    if num_samples_features != num_samples_flexion:\n        downsample_factor = num_samples_flexion // num_samples_features\n\n        #make nsure downsample_factor at least 1\n        downsample_factor = max(downsample_factor, 1)\n        downsize_data = decimate(train_data, downsample_factor, axis=0)\n\n        #make sure downsize_data has same number of samples as R_matrix\n        if downsize_data.shape[0] > num_samples_features:\n            downsize_data = downsize_data[:num_samples_features]\n    else:\n        downsize_data = train_data\n\n    #linear filter f using matrix multiplication\n    #p1 = (R^T R)^(-1)\n    p1 = np.linalg.inv(np.matmul(R_matrix.T, R_matrix))\n\n    #p2 = R^T Y\n    p2 = np.matmul(R_matrix.T, downsize_

In [ ]:
def linear_filter(R_matrix, train_data, lambda_reg=0.01):
    num_samples_features = R_matrix.shape[0]
    num_samples_flexion = train_data.shape[0]

    # Downsample train_data if it doesn't match the number of feature windows in R_matrix
    if num_samples_features != num_samples_flexion:
        downsample_factor = num_samples_flexion // num_samples_features
        downsample_factor = max(downsample_factor, 1)  # Ensure downsample_factor at least 1
        downsize_data = decimate(train_data, downsample_factor, axis=0)

        # Ensure downsize_data has same number of samples as R_matrix
        if downsize_data.shape[0] > num_samples_features:
            downsize_data = downsize_data[:num_samples_features]
    else:
        downsize_data = train_data

    # Regularized linear filter f using matrix multiplication
    # p1 = (R^T R + lambda * I)^(-1)
    lambda_identity = lambda_reg * np.eye(R_matrix.shape[1])  # Lambda times identity matrix
    p1 = np.linalg.inv(np.matmul(R_matrix.T, R_matrix) + lambda_identity)

    # p2 = R^T Y
    p2 = np.matmul(R_matrix.T, downsize_data)

    # f = p1 * p2
    lf = np.matmul(p1, p2)

    return downsize_data, lf

In [ ]:
#assign test ecog & glove data
test_ecog_1 = test_ecog[0]
test_ecog_2 = test_ecog[1]
test_ecog_3 = test_ecog[2]

test_glove_1 = test_dg[0]
test_glove_2 = test_dg[1]
test_glove_3 = test_dg[2]

In [ ]:
#feature extraction of test set
test_feature_1 = get_windowed_feats(test_ecog_1, 1000, 100, 50)
test_feature_2 = get_windowed_feats(test_ecog_2, 1000, 100, 50)
test_feature_3 = get_windowed_feats(test_ecog_3, 1000, 100, 50)

100%|██████████| 1799/1799 [00:01<00:00, 1300.85it/s]


In [ ]:
#R matrix for test set
test_R_1 = create_R_matrix(test_feature_1, N=3)
test_R_2 = create_R_matrix(test_feature_2, N=3)
test_R_3 = create_R_matrix(test_feature_3, N=3)

In [ ]:
"""mse = mean_squared_error(target_1, ensemble_preds)
r2 = r2_score(target_1, ensemble_preds)
print("MSE:", mse)
print("R^2:", r2)"""

'mse = mean_squared_error(target_1, ensemble_preds)\nr2 = r2_score(target_1, ensemble_preds)\nprint("MSE:", mse)\nprint("R^2:", r2)'

In [ ]:
multi_preds_1 = lgb_multioutput1.predict(test_R_1[:, 1:])
multi_preds_2 = lgb_multioutput2.predict(test_R_2[:, 1:])
multi_preds_3 = lgb_multioutput3.predict(test_R_3[:, 1:])

In [ ]:
lgb_preds_1 = lgb_model1.predict(test_R_1[:, 1:])
lgb_preds_2 = lgb_model2.predict(test_R_2[:, 1:])
lgb_preds_3 = lgb_model3.predict(test_R_3[:, 1:])

AttributeError: 'list' object has no attribute 'predict'

In [ ]:
xgb_preds_1 = xgb_model1.predict(test_R_1[:, 1:])
xgb_preds_2 = xgb_model2.predict(test_R_2[:, 1:])
xgb_preds_3 = xgb_model3.predict(test_R_3[:, 1:])

cat_preds_1 = cat_model1.predict(test_R_1[:, 1:])
cat_preds_2 = cat_model2.predict(test_R_2[:, 1:])
cat_preds_3 = cat_model3.predict(test_R_3[:, 1:])

"""lgb_preds_1 = lgb_model1.predict(test_R_1[:, 1:])
lgb_preds_2 = lgb_model2.predict(test_R_2[:, 1:])
lgb_preds_3 = lgb_model3.predict(test_R_3[:, 1:])"""

'lgb_preds_1 = lgb_model1.predict(test_R_1[:, 1:])\nlgb_preds_2 = lgb_model2.predict(test_R_2[:, 1:])\nlgb_preds_3 = lgb_model3.predict(test_R_3[:, 1:])'

In [ ]:
#linear filter
test_glove_ds_1, tf_1 = linear_filter(test_R_1, test_glove_1)
test_glove_ds_2, tf_2 = linear_filter(test_R_2, test_glove_2)
test_glove_ds_3, tf_3 = linear_filter(test_R_3, test_glove_3)

In [ ]:
#generate prediction of linear f
y_pred_1 = np.dot(test_R_1, tf_1)
y_pred_2 = np.dot(test_R_2, tf_2)
y_pred_3 = np.dot(test_R_3, tf_3)

In [ ]:
#correlation using pearsonr
def calculate_correlations(y_true, y_pred):
    correlations = []
    for finger in range(y_true.shape[1]):  # Assuming y_true.shape[1] is the number of fingers
        corr, _ = pearsonr(y_true[:, finger], y_pred[:, finger])
        correlations.append(corr)
    return correlations

In [ ]:
correlations_multi_1 = calculate_correlations(test_glove_ds_1, multi_preds_1)
correlations_multi_2 = calculate_correlations(test_glove_ds_2, multi_preds_2)
correlations_multi_3 = calculate_correlations(test_glove_ds_3, multi_preds_3)

In [ ]:
#correlations
correlations_xgb_1 = calculate_correlations(test_glove_ds_1, xgb_preds_1)
correlations_xgb_2 = calculate_correlations(test_glove_ds_2, xgb_preds_2)
correlations_xgb_3 = calculate_correlations(test_glove_ds_3, xgb_preds_3)

correlations_cat_1 = calculate_correlations(test_glove_ds_1, cat_preds_1)
correlations_cat_2 = calculate_correlations(test_glove_ds_2, cat_preds_2)
correlations_cat_3 = calculate_correlations(test_glove_ds_3, cat_preds_3)

In [ ]:
correlations_multi_1, correlations_multi_2, correlations_multi_3

In [ ]:
#create function for correlation
def cor(test_data, f, R_matrix):
    """
    Calculates Pearson correlation coefficients between true and predicted finger angles.

    Parameters:
    - test_data: The actual finger flexion data for the test set (samples x fingers).
    - f: The linear filter coefficients (features x fingers) calculated from the training set.
    - R_matrix: The feature matrix from the test ECoG data (samples x features).

    Returns:
    - A list of Pearson correlation coefficients, one for each finger.
    """
    #predict finger angles
    predicted = np.dot(R_matrix, f)

    #make sure predicted data has same shape as test data
    assert predicted.shape == test_data.shape, "Shape mismatch between predicted and true data"

    #Pearson correlation for each finger
    subject_correlations = []
    for finger in range(test_data.shape[1]):
        true_angle = test_data[:, finger]
        predicted_angle = predicted[:, finger]

        #correlation
        correlation = np.corrcoef(true_angle, predicted_angle)[0, 1]

        #append correlation coefficient
        subject_correlations.append(correlation)

    return subject_correlations

In [ ]:
#downsample glove data
train_glove_ds_1, f_1 = linear_filter(R_1, train_glove_1)
train_glove_ds_2, f_2 = linear_filter(R_2, train_glove_2)
train_glove_ds_3, f_3 = linear_filter(R_3, train_glove_3)

In [ ]:
"""correlations_lf_1, correlations_lf_2, correlations_lf_3"""

In [ ]:
"""#correlation
co1 = cor(train_glove_ds_1, f_1, R_1)
co2 = cor(train_glove_ds_2, f_2, R_2)
co3 = cor(train_glove_ds_3, f_3, R_3)

(co1, co2, co3)"""

**XGBoost**

In [ ]:
(correlations_xgb_1, correlations_xgb_2, correlations_xgb_3)

([0.42232087362320864,
  0.5792825705548416,
  0.1375833511080457,
  0.4515971698723468,
  0.11280150058714171],
 [0.5085427665095934,
  0.17824688508863096,
  0.2125406013319665,
  0.5175084733145717,
  0.07650407718577555],
 [0.5698591901088298,
  0.3808595203763699,
  0.4531288289641911,
  0.5035554827206609,
  0.36204981936624525])

In [ ]:
"""correlations_xgb_m1, correlations_xgb_m2, correlations_xgb_m3"""

**CatBoost**

In [ ]:
(correlations_cat_1, correlations_cat_2, correlations_cat_3)

([0.4515281141722916,
  0.6100341432652044,
  0.17987936346454056,
  0.49870862185773174,
  0.11510920928297969],
 [0.49312217446906437,
  0.19973550922066757,
  0.28184504641469527,
  0.4870203260072296,
  0.33360122641518797],
 [0.5867612865384663,
  0.38391599391965514,
  0.516004225557485,
  0.5726448589428662,
  0.43779103958155086])

In [ ]:
"""correlations_cat_m1, correlations_cat_m2, correlations_cat_m3"""

**LGBoost**

In [ ]:
(correlations_lgb_1, correlations_lgb_2, correlations_lgb_3)

NameError: name 'correlations_lgb_1' is not defined

In [ ]:
!jupyter nbconvert --to html final_project_part_2.ipynb